In [1]:
import numpy as np
import pandas as pd
import pickle
from tensorflow.keras.models import load_model


In [2]:
# Load the model created in the previous step
model = load_model('customer_churn_model.h5') 

In [3]:
# Load the scaler, label encoder and one hot encoder
scaler = pickle.load(open('scaler.pkl', 'rb'))
label_encoder = pickle.load(open('gender_label_encoder.pkl', 'rb'))
onehot_encoder = pickle.load(open('geography_one_hot_encoder.pkl', 'rb'))

In [5]:
# Input data for prediction
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [6]:
# Apply the same preprocessing steps to input data as in the training phase

In [7]:
## One hot encode the geography
geography = onehot_encoder.transform(np.array(input_data['Geography']).reshape(-1, 1)).toarray()
geography

array([[1., 0., 0.]])

In [13]:
# Create a dataframe from the one hot encoded geography
geography_df = pd.DataFrame(geography, columns=onehot_encoder.get_feature_names_out(['Geography']))

In [14]:
geography_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [16]:
# Create a dataframe from the input data
input_data_df = pd.DataFrame([input_data])
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [17]:
# Label encode the Gender
input_data_df['Gender'] = label_encoder.transform(input_data_df['Gender'])
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [18]:
# Concatenate the input data and the one hot encoded geography
input_data_df = pd.concat([input_data_df, geography_df], axis=1)

# Drop the Geography column
input_data_df.drop('Geography', axis=1, inplace=True)

input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [19]:
# Scale the input data
input_data_scaled = scaler.transform(input_data_df)
input_data_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [20]:
# Make predictions
predictions = model.predict(input_data_scaled)
predictions

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step


array([[0.03016577]], dtype=float32)

In [22]:
# Get the prediction probability
churn_probability = predictions[0][0]

In [24]:
print(churn_probability)

0.030165771


In [25]:
if churn_probability > 0.5:
    print("This customer is likely to churn")
else:
    print("This customer is not likely to churn")

This customer is not likely to churn
